# Reconhecimento Facial utilizando um modelo pré-treinado


A implementação é inspirada em dois artigos inovadores sobre reconhecimento facial usando redes neurais convolucionais, nomeadas de FaceNet e DeepFace.

Foi utilizado o modelo pré-treinado Keras-OpenFace , que é uma implementação open source do OpenFace no Keras (Implementado originalmente no PyTorch)

O modelo pré-treinado utilizado foi a implementação do Victor Sy Wang's, e foi carregado com o seguinte código:
https://github.com/iwantooxxoox/Keras-OpenFace.

# Importação das bibliotecas

Importar o arquivo utils.py do https://github.com/iwantooxxoox/Keras-OpenFace/blob/master/utils.py no qual contém funções utéis para a criação da rede neural e carregar seus pesos.

In [1]:
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.engine.topology import Layer
from keras import backend as K
from mtcnn.mtcnn import MTCNN
from matplotlib import pyplot
import cv2
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
from utils import LRN2D
import utils

%load_ext autoreload
%autoreload 2

#np.set_printoptions(threshold=np.nan)

Using TensorFlow backend.


# Construção da Rede Neural

O modelo implementado é baseado no modelo FaceNet.

A implementação do modelo está disponível aqui: https://github.com/iwantooxxoox/Keras-OpenFace

In [2]:
myInput = Input(shape=(96, 96, 3))

x = ZeroPadding2D(padding=(3, 3), input_shape=(96, 96, 3))(myInput)
x = Conv2D(64, (7, 7), strides=(2, 2), name='conv1')(x)
x = BatchNormalization(axis=3, epsilon=0.00001, name='bn1')(x)
x = Activation('relu')(x)
x = ZeroPadding2D(padding=(1, 1))(x)
x = MaxPooling2D(pool_size=3, strides=2)(x)
x = Lambda(LRN2D, name='lrn_1')(x)
x = Conv2D(64, (1, 1), name='conv2')(x)
x = BatchNormalization(axis=3, epsilon=0.00001, name='bn2')(x)
x = Activation('relu')(x)
x = ZeroPadding2D(padding=(1, 1))(x)
x = Conv2D(192, (3, 3), name='conv3')(x)
x = BatchNormalization(axis=3, epsilon=0.00001, name='bn3')(x)
x = Activation('relu')(x)
x = Lambda(LRN2D, name='lrn_2')(x)
x = ZeroPadding2D(padding=(1, 1))(x)
x = MaxPooling2D(pool_size=3, strides=2)(x)

# Inception3a
inception_3a_3x3 = Conv2D(96, (1, 1), name='inception_3a_3x3_conv1')(x)
inception_3a_3x3 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_3x3_bn1')(inception_3a_3x3)
inception_3a_3x3 = Activation('relu')(inception_3a_3x3)
inception_3a_3x3 = ZeroPadding2D(padding=(1, 1))(inception_3a_3x3)
inception_3a_3x3 = Conv2D(128, (3, 3), name='inception_3a_3x3_conv2')(inception_3a_3x3)
inception_3a_3x3 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_3x3_bn2')(inception_3a_3x3)
inception_3a_3x3 = Activation('relu')(inception_3a_3x3)

inception_3a_5x5 = Conv2D(16, (1, 1), name='inception_3a_5x5_conv1')(x)
inception_3a_5x5 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_5x5_bn1')(inception_3a_5x5)
inception_3a_5x5 = Activation('relu')(inception_3a_5x5)
inception_3a_5x5 = ZeroPadding2D(padding=(2, 2))(inception_3a_5x5)
inception_3a_5x5 = Conv2D(32, (5, 5), name='inception_3a_5x5_conv2')(inception_3a_5x5)
inception_3a_5x5 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_5x5_bn2')(inception_3a_5x5)
inception_3a_5x5 = Activation('relu')(inception_3a_5x5)

inception_3a_pool = MaxPooling2D(pool_size=3, strides=2)(x)
inception_3a_pool = Conv2D(32, (1, 1), name='inception_3a_pool_conv')(inception_3a_pool)
inception_3a_pool = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_pool_bn')(inception_3a_pool)
inception_3a_pool = Activation('relu')(inception_3a_pool)
inception_3a_pool = ZeroPadding2D(padding=((3, 4), (3, 4)))(inception_3a_pool)

inception_3a_1x1 = Conv2D(64, (1, 1), name='inception_3a_1x1_conv')(x)
inception_3a_1x1 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_1x1_bn')(inception_3a_1x1)
inception_3a_1x1 = Activation('relu')(inception_3a_1x1)

inception_3a = concatenate([inception_3a_3x3, inception_3a_5x5, inception_3a_pool, inception_3a_1x1], axis=3)

# Inception3b
inception_3b_3x3 = Conv2D(96, (1, 1), name='inception_3b_3x3_conv1')(inception_3a)
inception_3b_3x3 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_3x3_bn1')(inception_3b_3x3)
inception_3b_3x3 = Activation('relu')(inception_3b_3x3)
inception_3b_3x3 = ZeroPadding2D(padding=(1, 1))(inception_3b_3x3)
inception_3b_3x3 = Conv2D(128, (3, 3), name='inception_3b_3x3_conv2')(inception_3b_3x3)
inception_3b_3x3 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_3x3_bn2')(inception_3b_3x3)
inception_3b_3x3 = Activation('relu')(inception_3b_3x3)

inception_3b_5x5 = Conv2D(32, (1, 1), name='inception_3b_5x5_conv1')(inception_3a)
inception_3b_5x5 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_5x5_bn1')(inception_3b_5x5)
inception_3b_5x5 = Activation('relu')(inception_3b_5x5)
inception_3b_5x5 = ZeroPadding2D(padding=(2, 2))(inception_3b_5x5)
inception_3b_5x5 = Conv2D(64, (5, 5), name='inception_3b_5x5_conv2')(inception_3b_5x5)
inception_3b_5x5 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_5x5_bn2')(inception_3b_5x5)
inception_3b_5x5 = Activation('relu')(inception_3b_5x5)

inception_3b_pool = Lambda(lambda x: x**2, name='power2_3b')(inception_3a)
inception_3b_pool = AveragePooling2D(pool_size=(3, 3), strides=(3, 3))(inception_3b_pool)
inception_3b_pool = Lambda(lambda x: x*9, name='mult9_3b')(inception_3b_pool)
inception_3b_pool = Lambda(lambda x: K.sqrt(x), name='sqrt_3b')(inception_3b_pool)
inception_3b_pool = Conv2D(64, (1, 1), name='inception_3b_pool_conv')(inception_3b_pool)
inception_3b_pool = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_pool_bn')(inception_3b_pool)
inception_3b_pool = Activation('relu')(inception_3b_pool)
inception_3b_pool = ZeroPadding2D(padding=(4, 4))(inception_3b_pool)

inception_3b_1x1 = Conv2D(64, (1, 1), name='inception_3b_1x1_conv')(inception_3a)
inception_3b_1x1 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_1x1_bn')(inception_3b_1x1)
inception_3b_1x1 = Activation('relu')(inception_3b_1x1)

inception_3b = concatenate([inception_3b_3x3, inception_3b_5x5, inception_3b_pool, inception_3b_1x1], axis=3)

# Inception3c
inception_3c_3x3 = utils.conv2d_bn(inception_3b,
                                   layer='inception_3c_3x3',
                                   cv1_out=128,
                                   cv1_filter=(1, 1),
                                   cv2_out=256,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(2, 2),
                                   padding=(1, 1))

inception_3c_5x5 = utils.conv2d_bn(inception_3b,
                                   layer='inception_3c_5x5',
                                   cv1_out=32,
                                   cv1_filter=(1, 1),
                                   cv2_out=64,
                                   cv2_filter=(5, 5),
                                   cv2_strides=(2, 2),
                                   padding=(2, 2))

inception_3c_pool = MaxPooling2D(pool_size=3, strides=2)(inception_3b)
inception_3c_pool = ZeroPadding2D(padding=((0, 1), (0, 1)))(inception_3c_pool)

inception_3c = concatenate([inception_3c_3x3, inception_3c_5x5, inception_3c_pool], axis=3)

#inception 4a
inception_4a_3x3 = utils.conv2d_bn(inception_3c,
                                   layer='inception_4a_3x3',
                                   cv1_out=96,
                                   cv1_filter=(1, 1),
                                   cv2_out=192,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(1, 1),
                                   padding=(1, 1))
inception_4a_5x5 = utils.conv2d_bn(inception_3c,
                                   layer='inception_4a_5x5',
                                   cv1_out=32,
                                   cv1_filter=(1, 1),
                                   cv2_out=64,
                                   cv2_filter=(5, 5),
                                   cv2_strides=(1, 1),
                                   padding=(2, 2))

inception_4a_pool = Lambda(lambda x: x**2, name='power2_4a')(inception_3c)
inception_4a_pool = AveragePooling2D(pool_size=(3, 3), strides=(3, 3))(inception_4a_pool)
inception_4a_pool = Lambda(lambda x: x*9, name='mult9_4a')(inception_4a_pool)
inception_4a_pool = Lambda(lambda x: K.sqrt(x), name='sqrt_4a')(inception_4a_pool)
inception_4a_pool = utils.conv2d_bn(inception_4a_pool,
                                   layer='inception_4a_pool',
                                   cv1_out=128,
                                   cv1_filter=(1, 1),
                                   padding=(2, 2))
inception_4a_1x1 = utils.conv2d_bn(inception_3c,
                                   layer='inception_4a_1x1',
                                   cv1_out=256,
                                   cv1_filter=(1, 1))
inception_4a = concatenate([inception_4a_3x3, inception_4a_5x5, inception_4a_pool, inception_4a_1x1], axis=3)

#inception4e
inception_4e_3x3 = utils.conv2d_bn(inception_4a,
                                   layer='inception_4e_3x3',
                                   cv1_out=160,
                                   cv1_filter=(1, 1),
                                   cv2_out=256,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(2, 2),
                                   padding=(1, 1))
inception_4e_5x5 = utils.conv2d_bn(inception_4a,
                                   layer='inception_4e_5x5',
                                   cv1_out=64,
                                   cv1_filter=(1, 1),
                                   cv2_out=128,
                                   cv2_filter=(5, 5),
                                   cv2_strides=(2, 2),
                                   padding=(2, 2))
inception_4e_pool = MaxPooling2D(pool_size=3, strides=2)(inception_4a)
inception_4e_pool = ZeroPadding2D(padding=((0, 1), (0, 1)))(inception_4e_pool)

inception_4e = concatenate([inception_4e_3x3, inception_4e_5x5, inception_4e_pool], axis=3)

#inception5a
inception_5a_3x3 = utils.conv2d_bn(inception_4e,
                                   layer='inception_5a_3x3',
                                   cv1_out=96,
                                   cv1_filter=(1, 1),
                                   cv2_out=384,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(1, 1),
                                   padding=(1, 1))

inception_5a_pool = Lambda(lambda x: x**2, name='power2_5a')(inception_4e)
inception_5a_pool = AveragePooling2D(pool_size=(3, 3), strides=(3, 3))(inception_5a_pool)
inception_5a_pool = Lambda(lambda x: x*9, name='mult9_5a')(inception_5a_pool)
inception_5a_pool = Lambda(lambda x: K.sqrt(x), name='sqrt_5a')(inception_5a_pool)
inception_5a_pool = utils.conv2d_bn(inception_5a_pool,
                                   layer='inception_5a_pool',
                                   cv1_out=96,
                                   cv1_filter=(1, 1),
                                   padding=(1, 1))
inception_5a_1x1 = utils.conv2d_bn(inception_4e,
                                   layer='inception_5a_1x1',
                                   cv1_out=256,
                                   cv1_filter=(1, 1))

inception_5a = concatenate([inception_5a_3x3, inception_5a_pool, inception_5a_1x1], axis=3)

#inception_5b
inception_5b_3x3 = utils.conv2d_bn(inception_5a,
                                   layer='inception_5b_3x3',
                                   cv1_out=96,
                                   cv1_filter=(1, 1),
                                   cv2_out=384,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(1, 1),
                                   padding=(1, 1))
inception_5b_pool = MaxPooling2D(pool_size=3, strides=2)(inception_5a)
inception_5b_pool = utils.conv2d_bn(inception_5b_pool,
                                   layer='inception_5b_pool',
                                   cv1_out=96,
                                   cv1_filter=(1, 1))
inception_5b_pool = ZeroPadding2D(padding=(1, 1))(inception_5b_pool)

inception_5b_1x1 = utils.conv2d_bn(inception_5a,
                                   layer='inception_5b_1x1',
                                   cv1_out=256,
                                   cv1_filter=(1, 1))
inception_5b = concatenate([inception_5b_3x3, inception_5b_pool, inception_5b_1x1], axis=3)

av_pool = AveragePooling2D(pool_size=(3, 3), strides=(1, 1))(inception_5b)
reshape_layer = Flatten()(av_pool)
dense_layer = Dense(128, name='dense_layer')(reshape_layer)
norm_layer = Lambda(lambda  x: K.l2_normalize(x, axis=1), name='norm_layer')(dense_layer)


# Final Model
model = Model(inputs=[myInput], outputs=norm_layer)

# Carregar o modelo com pesos pré-treinados

A FaceNet é treinado minimizado o triplet loss. Será carregado um modelo previamente treinado.

In [3]:
weights = utils.weights
weights_dict = utils.load_weights()

# Set layer weights of the model
for name in weights:
  if model.get_layer(name) != None:
    model.get_layer(name).set_weights(weights_dict[name])
  elif model.get_layer(name) != None:
    model.get_layer(name).set_weights(weights_dict[name])

## Função <font color=blue>image_to_embedding</font>

Quando o modelo é carregado com pesos pré-treinados, pode-se criar os **128 vetores dimensionais embedding** para todas as faces salvas na pasta "images".

A função **"image_to_embedding"** passa uma imagem pro modelo Inception, para gerar os vetores embedding.

In [4]:
def image_to_embedding(image, model):
    #image = cv2.resize(image, (96, 96), interpolation=cv2.INTER_AREA) 
    image = cv2.resize(image, (96, 96)) 
    img = image[...,::-1]
    img = np.around(np.transpose(img, (0,1,2))/255.0, decimals=12)
    x_train = np.array([img])
    embedding = model.predict_on_batch(x_train)
    return embedding

## Função <font color=blue>recognize_face</font>

Essa função calcula a similaridade entre a imagem capturado e as imagens que já estão armazenadas. Passa a imagem para o modelo de rede neural pré-treinado para gerar o vetor embedding. Este vetor será comparado com todos os outros vetores embeddings das imagens salvas, por meio da distância euclidiana L2.

Se a mínima distância de L2 entre dois vetores embedding for menor do que o treshold (aqui o valor de treshold foi de .68, podendo ser ajustado) então serão correspondentes.

In [5]:
def recognize_face(face_image, input_embeddings, model):

    embedding = image_to_embedding(face_image, model)
    
    minimum_distance = 200
    name = None
    
    for(input_name, input_embedding) in input_embeddings.items():    
        euclidean_distance = np.linalg.norm(embedding-input_embedding)
#         print('Euclidean distance from %s is %s' %(input_name, euclidean_distance))

        if(euclidean_distance < minimum_distance):
            minimum_distance = euclidean_distance
            name = input_name
    
    if(minimum_distance < 0.68):
        return str(name)
    else:
        return None

## Função <font color=blue>create_input_image_embeddings</font>

Essa função gera imagens embedding com dimensão de 128 para todas as imagens salvas na pasta "images" atráves da rede neural treinada. Cria um dicionário com o nome da imagem sendo a chave e os valores sendo o embedding.

## Função <font color=blue>recognize_faces_in_cam</font>

Essa função captura a imagem através da webcam, detectar a face e cortar a imagem para ter somente a face, em seguida é passada para a função <font color=blue>recognize_face</font>.

In [6]:
import glob
import re

def create_input_image_embeddings():
    input_embeddings = {}

    for file in glob.glob("images/*/*.jpg"):
        person_name = os.path.splitext(os.path.basename(file))[0]
        person_name = re.search(r'(\D+)\_', person_name)
        person_name = person_name.group(1)
        image_file = cv2.imread(file, 1)
        input_embeddings[person_name] = image_to_embedding(image_file, model)

    return input_embeddings

def recognize_faces_in_cam(input_embeddings):
    

    cv2.namedWindow("Face Recognizer")
    vc = cv2.VideoCapture(0)
   

    font = cv2.FONT_HERSHEY_SIMPLEX
    
    # Cascade Classifier OpenCV
#     face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    
    #MultiTask CNN
    face_detector = MTCNN()
    
    while vc.isOpened():
        _, frame = vc.read()
        img = frame
        height, width, channels = frame.shape
        
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
#         faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        faces = face_detector.detect_faces(img)
        
        # Loop through all the faces detected 
        identities = []
#         for (x, y, w, h) in faces:
#             x1 = x
#             y1 = y
#             x2 = x+w
#             y2 = y+h

        for i in range(len(faces)):
            x1, y1, w, h = faces[i]['box']
            x2, y2 = x1 + w, y1 + h
            
            face_image = frame[max(0, y1):min(height, y2), max(0, x1):min(width, x2)]    
            identity = recognize_face(face_image, input_embeddings, model)

            if identity is not None:
                img = cv2.rectangle(frame,(x1, y1),(x2, y2),(255,255,255),2)
                cv2.putText(img, str(identity), (x1+5,y1-5), font, 1, (255,255,255), 2)
        
        key = cv2.waitKey(100)
        cv2.imshow("Face Recognizer", img)

        if key == 27: #saída no ESC
            break
    vc.release()
    cv2.destroyAllWindows()

## Interface Tkinter

In [7]:
import tkinter as tk 

dir_name = ""

def start_app():    
    master = tk.Tk()
    master.title("Smart Glass")

    tk.Label(master, text='Nome').grid(row=0) 
    tk.Label(master, text='Sobrenome').grid(row=1) 

    name = tk.Entry(master) 
    last_name = tk.Entry(master) 
    name.grid(row=0, column=1) 
    last_name.grid(row=1, column=1) 

    def callback(event=None):
        global dir_name
        dir_name = name.get() + last_name.get()
        master.destroy()

    btn = tk.Button(master, text = "OK", width=10, command = callback)
    btn.grid(row=2, column=1)

    master.bind("<Return>", callback)
    master.lift()
    master.mainloop()

## Capturar faces

O seguinte código captura 10 imagens de uma pessoa. Elas são salvas na pasta "images" com o nome de User_1 até User_10. Selecione a melhor imagem das 10. Renomeie a foto com o nome dela, e apague o resto das imagens. Essa imagem será utilizada para reconhecer a face da pessoa através do One Shot Learning.

### Cascade Classifier OpenCV

In [20]:
cam = cv2.VideoCapture(0)

face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

count = 0
while(True):
    ret, img = cam.read()
    faces = face_detector.detectMultiScale(img, 1.3, 5)
    for (x,y,w,h) in faces:
        x1 = x
        y1 = y
        x2 = x+w
        y2 = y+h
        cv2.rectangle(img, (x1,y1), (x2,y2), (255,255,255), 2)     
        count += 1
        # Salvar as imagens dentro da pasta 'images'
        cv2.imwrite("images/User_" + str(count) + ".jpg", img[y1:y2,x1:x2])
        cv2.imshow('image', img)
    k = cv2.waitKey(200) & 0xff #saída no ESC
    if k == 27:
        break
    elif count >= 10:
         break
cam.release()
cv2.destroyAllWindows()

### MultiTask CNN Classifier

In [9]:
start_app()

try:
    os.mkdir("images/{}/".format(dir_name))
except FileExistsError:
    pass

cam = cv2.VideoCapture(0)

face_detector = MTCNN()

count = 0
while(True):
    ret, img = cam.read()
    faces = face_detector.detect_faces(img)
    for i in range(len(faces)):
        x1, y1, width, height = faces[i]['box']
        x2, y2 = x1 + width, y1 + height
        cv2.rectangle(img, (x1,y1), (x2,y2), (255,255,255), 2)     
        count += 1
        # Salvar as imagens dentro da pasta 'images'
        cv2.imwrite("images/{}/{}_".format(dir_name, dir_name) + str(count) + ".jpg", img[y1:y2,x1:x2])
        cv2.imshow('image', img)
    k = cv2.waitKey(200) & 0xff #saída no ESC
    if k == 27:
        break
    elif count >= 10:
         break
cam.release()
cv2.destroyAllWindows()

In [10]:
input_embeddings = create_input_image_embeddings()

input_embeddings

{'DinhoSoares': array([[ 0.04802728,  0.03750252, -0.08263657,  0.00491131, -0.01768545,
          0.14761376, -0.06787279, -0.03890521, -0.0056197 , -0.13087645,
          0.08630715,  0.14676256,  0.0527203 , -0.11067671, -0.18464951,
          0.00869838, -0.08495291,  0.05536461, -0.14349858,  0.15115435,
          0.12760827, -0.03926608,  0.08891631,  0.02342255, -0.0901574 ,
          0.01725823,  0.05596548, -0.13844149, -0.13461116,  0.07922625,
         -0.06501531, -0.03172664,  0.08975697,  0.12704276, -0.02692638,
          0.01741591, -0.00755119,  0.12259854, -0.08028157, -0.0027083 ,
         -0.06212519, -0.07960817,  0.01564151,  0.09065685, -0.10148361,
         -0.08002614,  0.1612622 , -0.0888348 , -0.17039587,  0.21055299,
          0.06244998, -0.0820504 , -0.09251866,  0.12719445,  0.10222878,
          0.01857189, -0.02337919,  0.03063533, -0.06617044, -0.09275288,
         -0.05792993, -0.04702435,  0.21706247, -0.13601862,  0.11291526,
         -0.16156003, -

In [9]:
model.save("model_complete.h5")

In [9]:
model.save_weights('./model/model_weights.h5')

# Save the model architecture
with open('./model/model_architecture.json', 'w') as f:
    f.write(model.to_json())